Connect to the Ophidia server side

In [ ]:
from PyOphidia import Client, Cube
cli = Client(read_env = True)
Cube.setclient(cli)

In [ ]:
hosts = "1"
threads = "8"
partition = "partition"

Startup the computing components on the server-side

In [ ]:
Cube.cluster(action = 'deploy', host_partition = partition, nhost = hosts, exec_mode = "async")

Load a NetCDF file. The file, produced by CMCC from the CMCC-ESM2 model, can be freely downloaded from the CMIP6 archive (https://esgf-node.llnl.gov/projects/cmip6/). 

In [ ]:
import urllib.request
filename, headers = urllib.request.urlretrieve('https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/ScenarioMIP/CMCC/CMCC-ESM2/ssp585/r1i1p1f1/day/tasmin/gn/v20210126/tasmin_day_CMCC-ESM2_ssp585_r1i1p1f1_gn_20900101-21001231.nc')

In [ ]:
myCube1 = Cube.importnc2(src_path = filename, measure = 'tasmin', imp_dim = 'time', description = 'Min Temperatures', host_partition = partition, nthreads = threads)

Apply a mask on each of the values of the imported datacube. 

In [ ]:
myCube2 = myCube1.apply(query = "oph_predicate2(measure,'x-293.15','>0','1','0')", measure_type = 'auto', nthreads = threads)

Perform a yearly aggregation of the previous datacube

In [ ]:
myCube3 = myCube2.reduce2(operation = 'sum', dim = 'time', concept_level = 'y', nthreads = threads)

Extract the first year from the previous datacube.

In [ ]:
myCube4 = myCube3.subset(subset_dims = "time", subset_filter = 1)

Retrieve data from the server-side datacube and convert it into a Xarray dataframe.

In [ ]:
pythonData = myCube4.to_dataset()

Plot the retrieved data using Xarray built-in capabilities based on Matplotlib

In [ ]:
pythonData.tasmin.plot(cmap = "Oranges", cbar_kwargs={"label": "Tropical Nights count"}, figsize = (16,8))

Shut down the computing components on the server-side

In [ ]:
Cube.cluster(action = 'undeploy', host_partition = partition, exec_mode = 'async')